# 2CSSID-TP01. Prétraitement

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from typing import Tuple

## I. Réalisation des algorithmes

Cette partie sert à améliorer la compréhension des algorithmes de préparation de données vus en
cours en les implémentant à partir de zéro. Pour ce faire, on va utiliser la bibliothèque numpy qui
est utile dans les calcules surtout matricielles.

### I.1. Normalisation

Ici, on va réaliser les deux fonctions de nomalisation : standard et min-max.
On va prendre une matrice $X[N, M]$ de $N$ échantillons et $M$ colonnes.
La normalisation standard d'une colonne $j$ peut être décrite comme : 
$$standard(X_j) = \frac{X_j - \mu(X_j)}{\sigma(X_j)}$$
La nomalisation min-max d'une colonne $j$ peut être décrite comme : 
$$minmax(X_j) = \frac{X_j - min(X_j)}{max(X_j) - min(X_j)}$$


In [ ]:
# Entrée : la matrice des données (N échantillons X  M caractéristiques)
# Sortie : vecteur de M moyennes, vecteur de M écart-types, une matrice normalisée
def norm_std(X: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:

    #axis 0 is par columns
    v_moy = X.mean(axis=0)
    v_ecart_type = np.sqrt(X.var(axis=0))

    return v_moy, v_ecart_type, (X - v_moy) / v_ecart_type


#=====================================================================
# TEST UNITAIRE
#=====================================================================
# (array([4. , 3. , 0.5]),
#  array([1.87082869, 2.        , 0.5       ]),
#  array([[ 1.60356745,  1.        , -1.        ],
#         [-1.06904497, -1.        ,  1.        ],
#         [-0.53452248,  1.        , -1.        ],
#         [ 0.        , -1.        ,  1.        ]]))
#---------------------------------------------------------------------

X = np.array([
    [7, 5, 0],
    [2, 1, 1],
    [3, 5, 0],
    [4, 1, 1],
])

norm_std(X)

(array([4. , 3. , 0.5]),
 array([1.87082869, 2.        , 0.5       ]),
 array([[ 1.60356745,  1.        , -1.        ],
        [-1.06904497, -1.        ,  1.        ],
        [-0.53452248,  1.        , -1.        ],
        [ 0.        , -1.        ,  1.        ]]))

In [ ]:
# Entrée : la matrice des données (N échantillons X  M caractéristiques)
# Sortie : vecteur de M max, vecteur de M min, une matrice normalisée
def norm_minmax(X: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:

    #axis 0 is par columns
    v_min = X.min(axis=0)
    v_max = X.max(axis=0)

    return v_max, v_min, (X - v_min) / (v_max - v_min)

#=====================================================================
# TEST UNITAIRE
#=====================================================================
# (array([7, 5, 1]),
#  array([2, 1, 0]),
#  array([[1. , 1. , 0. ],
#         [0. , 0. , 1. ],
#         [0.2, 1. , 0. ],
#         [0.4, 0. , 1. ]]))
#---------------------------------------------------------------------

X = np.array([
    [7, 5, 0],
    [2, 1, 1],
    [3, 5, 0],
    [4, 1, 1],
])

norm_minmax(X)

(array([7, 5, 1]),
 array([2, 1, 0]),
 array([[1. , 1. , 0. ],
        [0. , 0. , 1. ],
        [0.2, 1. , 0. ],
        [0.4, 0. , 1. ]]))

### I.2. Encodage One-Hot

Etant donné un vecteur $A[N]$ représentant une caractéristique nominale donnée, on veut encoder les valeurs en utilisant One-Hot. Pour faciliter la tâche, on vous donne l'algorithme détaillé : 
1. Trouver les valeurs uniques dans le vecteur $A$ ; on appele ça : un vocabulaire $V$
1. Créer une matrice $X[N, |V|] en recopiant le vecteur $V$ $N$ fois. Dans python, on peut recopier un vecteur en utilisant l'instruction : [V] * N
1. Comparer l'égalité entre chaque ligne de $A$ et chaque ligne (qui est un vecteur) de $X$.
1. Transformer les booléens vers des entiers

In [ ]:
# Entrée : un vecteur d'une caractéristique (N échantillons)
# Sortie : vecteur du vocabulaire V, matrice N X |V|

# Version utilisant l'algorithme proposé avec python
def one_hot(A: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    V = np.unique(A)
    X = [V]*len(A)

    X_bool = np.array([Xi == Ai for Xi,Ai in zip(X,A)])
    return V, X_bool.astype(int)

# Version utilisant les fonctions utilitaires de NumPy
def one_hot_np(A: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    V = np.unique(A)

    return V, (A[:, np.newaxis] == V).astype(int)

#=====================================================================
# TEST UNITAIRE
#=====================================================================
# (array(['COLD', 'HOT', 'MILD'], dtype='<U4'),
#  array([[0, 1, 0],
#         [0, 0, 1],
#         [1, 0, 0],
#         [0, 1, 0],
#         [0, 0, 1]]))
#---------------------------------------------------------------------

A = np.array(['HOT', 'MILD', 'COLD', 'HOT', 'MILD'])

# version utilisant l'algorithme proposé avec python
one_hot(A)

# version utilisant les fonctions utilitaires de NumPy
#one_hot_np(A)

(array(['COLD', 'HOT', 'MILD'], dtype='<U4'),
 array([[0, 1, 0],
        [0, 0, 1],
        [1, 0, 0],
        [0, 1, 0],
        [0, 0, 1]]))

### I.3. Binarisation

Etant donné un vecteur $A[N]$ représentant une caractéristique numérique donnée, on veut encoder les valeurs en 0 ou 1 selon un seuil $s$.
La binarization d'un élément $A_i$ est donnée par :
$$A_i' = \begin{cases}
1 & \text{si } A_i \ge s\\
0 & \text{sinon}\\
\end{cases}$$

In [ ]:
# Entrée : un vecteur d'une caractéristique (N échantillons), un nombre
# Sortie : un vecteur binarisé (N échantillons)
def bin(A: np.ndarray, seuil: float) -> np.ndarray:
    return (A >= seuil).astype(int)

#=====================================================================
# TEST UNITAIRE
#=====================================================================
# array([1, 0, 0, 0, 1, 1])
#---------------------------------------------------------------------

A = np.array([5, 2, 1, -1, 6, 4])

bin(A, 4)

array([1, 0, 0, 0, 1, 1])

## II. Application et analyse

Cette partie sert à appliquer les algorithmes, modifier les paramètres et analyser les résultats.

### II.1. Lecture des données

On va lire 4 fichiers : 
- un fichier CSV avec des colonnes séparées par des virgules
- un fichier CSV avec des colonnes séparées par des point-virgules
- un fichier Sqlite 
- un fichier XML

In [ ]:
adult1 = pd.read_csv("data/adult1.csv", skipinitialspace=True)
adult1.head(10)

,age,workclass,education,Marital-status,occupation,sex,Hours-per-week,class
0,39.0,State-gov,Bachelors,Never-married,Adm-clerical,Male,40,<=50K
1,50.0,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Male,13,<=50K
2,38.0,Private,HS-grad,Divorced,Handlers-cleaners,Male,40,<=50K
3,53.0,Private,11th,NaN,Handlers-cleaners,Male,40,<=50K
4,28.0,Private,Bachelors,Married-civ-spouse,Prof-specialty,Female,40,<=50K
5,37.0,Private,Masters,Married-civ-spouse,Exec-managerial,Female,40,<=50K
6,49.0,Private,9th,Married-spouse-absent,Other-service,Female,16,<=50K
7,52.0,Self-emp-not-inc,HS-grad,Married-civ-spouse,Exec-managerial,Male,45,>50K
8,31.0,Private,Masters,Never-married,Prof-specialty,Female,50,>50K
9,42.0,Private,Bachelors,Married-civ-spouse,Exec-managerial,Male,40,>50K


In [ ]:
noms = ["class", "age", "sex", "workclass", "education", "hours-per-week", "marital-status"]
adult2 = pd.read_csv("data/adult2.csv", skipinitialspace=True, sep=";", header=None, names=noms)
adult2.head(10)

,class,age,sex,workclass,education,hours-per-week,marital-status
0,N,25,F,Private,Some-college,40,Married-civ-spouse
1,N,18,F,Private,HS-grad,30,Never-married
2,Y,47,F,"Private, Prof-school",60,Married-civ-spouse,NaN
3,Y,50,M,Federal-gov,Bachelors,55,Divorced
4,N,47,M,Self-emp-inc,HS-grad,60,Divorced
5,Y,43,M,Private,Some-college,40,Married-civ-spouse
6,N,46,M,Private,5th-6th,40,Married-civ-spouse
7,N,35,M,Private,Assoc-voc,40,Married-civ-spouse
8,N,41,M,Private,HS-grad,48,Married-civ-spouse
9,"N,30",M,"Private, HS-grad",40,Married-civ-spouse,NaN,NaN


In [ ]:
import sqlite3
#établir la connexion avec la base de données
con = sqlite3.connect("data/adult3.db")
#récupérer le résultat d'une réquête SQL sur cette connexion
adult3 = pd.read_sql_query("SELECT * FROM income", con)

#remplacer les valeurs "?" par NaN de numpy
adult3 = adult3.replace('?', np.nan)

adult3.head(10)

,num,age,workclass,education,marital-status,sex,hours-per-day,class
0,1,76,Private,Masters,married,M,8.0,Y
1,2,44,Private,Bachelors,married,M,12.0,Y
2,3,47,Self-emp-not-inc,Masters,single,F,10.0,N
3,4,20,Private,Some-college,single,F,8.0,N
4,5,29,Private,HS-grad,single,M,8.0,N
5,6,32,Self-emp-inc,HS-grad,married,M,8.0,Y
6,7,17,NaN,10th,single,F,6.4,N
7,8,30,Private,11th,single,M,8.0,N
8,9,31,Local-gov,HS-grad,single,F,8.0,N
9,10,42,Private,HS-grad,married,M,8.0,N


In [ ]:
%pip install lxml


[notice] A new release of pip available: 22.2 -> 22.3
[notice] To update, run: C:\Users\walid\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from lxml import etree
#créer le parser et spécifier qu'il doit valider le DTD
parser = etree.XMLParser(dtd_validation=True)
#analyser le fichier XML en utilisant ce parser
arbre = etree.parse("data/adult4.xml", parser)

def valeur_noeud(noeud):
    return noeud.text if noeud is not None else np.nan

noms2 = ["id", "age", "workclass", "education", "marital-status", "sex", "hours-per-week", "class"]
adult4 = pd.DataFrame(columns=noms2)

for candidat in arbre.getroot():
    idi = candidat.get("id")
    age = valeur_noeud(candidat.find("age"))
    workclass = valeur_noeud(candidat.find("workclass"))
    education = valeur_noeud(candidat.find("education"))
    marital = valeur_noeud(candidat.find("marital-status"))
    sex = valeur_noeud(candidat.find("sex"))
    hours = valeur_noeud(candidat.find("hours-per-week"))
    klass = valeur_noeud(candidat.find("class"))

    adult4 = pd.concat(
        [adult4, 
         pd.Series([idi, age, workclass, education, marital, sex, hours, klass],index=noms2).to_frame().T
        ], axis=0, ignore_index=True)
adult4.head(10)

,id,age,workclass,education,marital-status,sex,hours-per-week,class
0,52,47,Local-gov,Some-college,divorced,F,38,N
1,53,34,Private,HS-grad,single,F,40,N
2,54,33,Private,Bachelors,single,F,40,N
3,55,21,Private,HS-grad,single,M,35,N
4,56,52,NaN,HS-grad,divorced,M,45,Y
5,57,48,Private,HS-grad,married,M,46,N
6,58,23,Private,Bachelors,single,M,40,N
7,59,71,Self-emp-not-inc,Some-college,divorced,M,2,N
8,60,29,Private,HS-grad,divorced,M,60,N
9,61,42,Private,Bachelors,divorced,M,50,N


**Analyse** 
- Que remarquez-vous concernant l'ordre, le nombre et les noms des caractéristiques dans les 4 datasets ?
- Que remarquez-vous à propos des valeurs dans les 4 tables ?

**Réponse**

d1: age,	workclass,	education,	Marital-status,	occupation,	    sex,	            Hours-per-week,	class,

d2: class,	age,	    sex,        workclass,	    education,	    hours-per-week, 	marital-status,

d3: num,	age,        workclass,	education,	    marital-status,	sex,	            hours-per-day,	class,

d4: id,	    age,        workclass,	education,	    marital-status,	sex,	            hours-per-week,	class,



- L'ordre des caractéristiqus n'est pas le même pour les quatres datasets, le nombre est différent (8 pour d1, 7 pour d2, 8 pour d3, 8 pour d4), les noms se ressemblent sauf pour la spécification l'unité (pour hours-per-unit), et la majuscule au debut du nom des caractéristiques (Hours-per-week and Martial-status)

- Les valeurs sont differentes a cause de l'unité de mesure (echèlle) (days , weeks pour Hours-per-unit) ou le type de representation (float , int pour age), ou les valeurs des catégories (M-Male, F-Female pour sex par exemple, et les differentes valeurs de marital-status, class qui représentent différemment les mêmes concepts: Married-civ-spouse, Married-spouse-absent et Married-AF-spouse representent "married"), ce phénomène s'appelle conflits de valeurs.

    De plus, on remarque l'existence de valeurs manquantes representées par NaN, et l'utilisation d'un mauvais séparateur a causé certains decalages de valeurs entre les colonnes ( une virgule a la place d'un point virgule par exemple dans la 2eme ligne de adulte 2)

### II.2. Intégration des données

Dans cette section, on va appliquer des opérations sur les différentes tables. Vous devez à chaque fois figurer ce qu'on a fait et pourquoi.

In [ ]:
# Afficher les noms des colonnes de adult3
list(adult3.columns)

['num',
 'age',
 'workclass',
 'education',
 'marital-status',
 'sex',
 'hours-per-day',
 'class']

In [ ]:
adult3.rename(columns={"num": "id", "hours-per-day": "hours-per-week"}, inplace=True)
adult1.rename(columns={"Hours-per-week": "hours-per-week", "Marital-status": "marital-status"}, inplace=True)

# Afficher les noms des colonnes de adult3
list(adult3.columns)

['id',
 'age',
 'workclass',
 'education',
 'marital-status',
 'sex',
 'hours-per-week',
 'class']

**Analyse** 
- Quelle opération a-t-on appliqué ?
- Pourquoi ? (Quel est l'intérêt ?)
- Est-ce qu'en appliquant cette opération, on aura certains problèmes ?

**Réponse**
- L'operation de renommage de caractéristiques (variables)
- Pour intégrer les schémas: identifier les différents noms des mêmes données réelles, pour ne pas dupliquer la même variable avec des noms differents.
- Oui, pour les variables quantitatives: on aura le problème d'unités différentes (echelles différentes), et pour les variables qualitatives: on aura des noms différents pour des classes qui modélisent le même concept.

In [ ]:
ordre = ["age", "workclass", "education", "marital-status", "sex", "hours-per-week", "class"]
adult1 = adult1.reindex(ordre + ["occupation"], axis=1)
adult2 = adult2.reindex(ordre, axis=1)
adult3 = adult3.reindex(ordre + ["id"], axis=1)
adult4 = adult4.reindex(ordre + ["id"], axis=1)

# Afficher les noms des colonnes de adult3
list(adult3.columns)

['age',
 'workclass',
 'education',
 'marital-status',
 'sex',
 'hours-per-week',
 'class',
 'id']

**Analyse** 
- Quelle opération a-t-on appliqué ?
- Pourquoi ? (Quel est l'intérêt ?)

**Réponse**
- Réordonnacement des colonnes des differentes tables pour suivre le même ordre pour les colonnes communes, plus les colonnes supplémentaires a la fin
- Pour superposer directement les colonnes ayant le meme nom lors de l'integration de données, et ajouter les colonnes spécifiques a un tableau donné. Ceci permet de concatener plus facilement les données dans la nouvelle table sans un traitement supplementaire.

In [ ]:
# Afficher les deux premières lignes de la table adult3
adult3.head(2)

,age,workclass,education,marital-status,sex,hours-per-week,class,id
0,76,Private,Masters,married,M,8.0,Y,1
1,44,Private,Bachelors,married,M,12.0,Y,2


In [ ]:
adult3["hours-per-week"] *= 5

# Afficher les deux premières lignes de la table adult3
adult3.head(2)

,age,workclass,education,marital-status,sex,hours-per-week,class,id
0,76,Private,Masters,married,M,40.0,Y,1
1,44,Private,Bachelors,married,M,60.0,Y,2


**Analyse** 
- Quelle opération a-t-on appliqué ?
- Pourquoi ? (Quel est l'intérêt ?)

**Réponse**
- Opération de mise à l'échelle (pour avoir la même unité). Pour ce cas, multiplier le nombre d'heures par jour fois 5 pour avoir le nombre d'heures par semaine.
- Pour résoudre les conflits de valeurs: avoir la même unité (l'unifier) pour que la concaténation des données soit correcte.

In [ ]:
adult34 = pd.concat([adult3, adult4], ignore_index=True)
adult34.head(10)

,age,workclass,education,marital-status,sex,hours-per-week,class,id
0,76,Private,Masters,married,M,40.0,Y,1
1,44,Private,Bachelors,married,M,60.0,Y,2
2,47,Self-emp-not-inc,Masters,single,F,50.0,N,3
3,20,Private,Some-college,single,F,40.0,N,4
4,29,Private,HS-grad,single,M,40.0,N,5
5,32,Self-emp-inc,HS-grad,married,M,40.0,Y,6
6,17,NaN,10th,single,F,32.0,N,7
7,30,Private,11th,single,M,40.0,N,8
8,31,Local-gov,HS-grad,single,F,40.0,N,9
9,42,Private,HS-grad,married,M,40.0,N,10


**Analyse** 
- Quelle opération a-t-on appliqué ?
- Pourquoi ? (Quel est l'intérêt ?)

**Réponse**
- L'operation de concatenation
- Pour combiner plusieurs sources de données et agrandir les datasets. (On regroupe les données dans un seul tableau pour faciliter toute analyse statistique sur ces dernières)

In [ ]:
# Transformer le champs "id" à un entier
adult34["id"] = pd.to_numeric(adult34["id"], downcast="integer")
# Ordonner la table en se basant sur les valeurs de "id"
adult34 = adult34.sort_values(by="id")

# L'opération que vous devez deviner (une opération de vérification)
red = adult34[adult34.duplicated("id", keep=False)]
red

,age,workclass,education,marital-status,sex,hours-per-week,class,id
44,70,Private,Some-college,single,M,40.0,N,45
94,70,Private,Some-college,single,M,40,N,45
45,31,Private,HS-grad,single,F,30.0,N,46
95,31,Private,HS-grad,single,NaN,30,N,46
46,22,Private,Some-college,married,M,24.0,N,47
96,22,Private,Some-college,married,M,24,N,47
47,36,Private,HS-grad,widowed,F,24.0,N,48
97,NaN,Private,HS-grad,widowed,F,24,N,48
48,64,Private,11th,married,M,40.0,N,49
98,64,Private,11th,married,M,40,N,49


**Analyse** 
- Quelle opération a-t-on appliqué ?
- Pourquoi ? (Quel est l'intérêt ?)

**Réponse**
- Affichage des champs ayant les mêmes identifiants (individus dupliqués) ordonnés selon leurs ids
- Pour identifier la redonance d'une même instance de données (le mème individu dupliqué) ainsi que les champs inexistants NaN en comparants entre les dupliqués d'une même instance. Par la suite, cette identification permet de ne garder qu'une seule instance par individu.

In [ ]:
# Il y a un problème avec cette forme
# en attendant qu'il soit réglé
#adult34 = adult34.groupby("id").ffill()

adult34.update(adult34.groupby(['id']).ffill())
adult34.update(adult34.groupby(['id']).bfill())

# L'opération de vérification précédente
red = adult34[adult34.duplicated("id", keep=False)]
red

,age,workclass,education,marital-status,sex,hours-per-week,class,id
44,70,Private,Some-college,single,M,40.0,N,45
94,70,Private,Some-college,single,M,40,N,45
45,31,Private,HS-grad,single,F,30.0,N,46
95,31,Private,HS-grad,single,F,30,N,46
46,22,Private,Some-college,married,M,24.0,N,47
96,22,Private,Some-college,married,M,24,N,47
47,36,Private,HS-grad,widowed,F,24.0,N,48
97,36,Private,HS-grad,widowed,F,24,N,48
48,64,Private,11th,married,M,40.0,N,49
98,64,Private,11th,married,M,40,N,49


**Analyse** 
- Quelle opération a-t-on appliqué ?
- Pourquoi ? (Quel est l'intérêt ?)

**Réponse**
- Remplissage des données manquantes NaN : comme le GroupBy va regrouper les dupliqués ensemble, appliquer un forward fill permet de remplir le champ NaN d'une colonne selon les cases précédentes sauf les premiers NaN rencontrés dans une colonne, c'est là où le backward fill va remplir les premières cases NaN par les suivantes.

- Pour régler le problème d'incohérence de données, et avoir un dataset plus précis et volumineux au lieu de supprimer les individus ayant de champs manquants. Graçe à la détection de la duplication d'individus, on peut remplir les champs manquants pour une analyse statistique plus correcte.

In [ ]:
adult34.drop_duplicates("id", keep="last", inplace=True)

# On refait la même opération précédente
red = adult34[adult34.duplicated("id", keep=False)]
red

,age,workclass,education,marital-status,sex,hours-per-week,class,id


**Analyse** 
- Quelle opération a-t-on appliqué ?
- Pourquoi ? (Quel est l'intérêt ?)

**Réponse**
- Supprimer les individus dupliqués et ne garder que la dernière occurence d'une instance de données dupliquée.
- Pour éliminer la redondance de données qui impace l'analyse statistique, car le même individu sera représenté plusieurs fois. Elle peut causer des poids disproportionnées lors de l'entraînement du modèle. La duplication de lignes peut également fausser les résultats d'un modèle car le dataset sera divisé en dataset: entraînement, validation et tests, et les données dupliquées peuvent être reparties sur les trois. D'où l'obtention d'un modèle biaisé et de mauvaises performances aprés deploiement, malgré le fait que l'evaluation du modèle l'a jugé efficace à cause de la duplication d'entrées.

In [ ]:
list(adult1.columns)

['age',
 'workclass',
 'education',
 'marital-status',
 'sex',
 'hours-per-week',
 'class',
 'occupation']

In [ ]:
adult1.drop(["occupation"], axis=1, inplace=True)
adult34.drop(["id"], axis=1, inplace=True)

list(adult1.columns)

['age',
 'workclass',
 'education',
 'marital-status',
 'sex',
 'hours-per-week',
 'class']

**Analyse** 
- Quelle opération a-t-on appliqué ?
- Pourquoi ? (Quel est l'intérêt ?)

**Réponse**
- Suppression des colonnes non communes entre les quatres tables.
- Pour préparer les tables à une autre concaténation en unifiant les schémas des quatre tables. Ceci permet de nettoyer la structure des données et faciliter l'apprentissage.

In [ ]:
# les différentes valeurs du colonne adult1.marital-status
adult1["marital-status"].unique()

array(['Never-married', 'Married-civ-spouse', 'Divorced', nan,
       'Married-spouse-absent', 'Separated', 'Married-AF-spouse'],
      dtype=object)

In [ ]:
dic = {
    "Never-married": "single",
    "Married-civ-spouse": "married",
    "Married-spouse-absent": "married",
    "Married-AF-spouse": "married",
    "Divorced": "divorced",
    "Separated": "divorced",
    "Widowed": "widowed"
}
adult1["marital-status"] = adult1["marital-status"].map(dic)
adult2["marital-status"] = adult2["marital-status"].map(dic)

# les différentes valeurs du colonne adult1.marital-status après mappage
adult1["marital-status"].unique()

array(['single', 'married', 'divorced', nan], dtype=object)

**Analyse** 
- Quelle opération a-t-on appliqué ?
- Pourquoi ? (Quel est l'intérêt ?)

**Réponse**
- Unification des valeurs de la variable qualitative marital-status qui representes le même concept avec un dictionnaire Python.
- Pour unifier les noms de classes des variables catégorielles communes entre les tables. Cette unification des valeurs des catégories résout le phénomène des conflits de valeurs, regroupe les valeurs qui represetent le même concept, et reduit également le nombre de catégories d'une variable. Ceci permet d'améliorer les performances du modèle car ses entrées seront cohérentes.

In [ ]:
# On va appliquer la même opération sur d'autres caractéristiques
adult1["sex"] = adult1["sex"].map({"Female": "F", "Male": "M"})
adult1["class"] = adult1["class"].map({"<=50K": "N", ">50K": "Y"})

# Ensuite, on fusionne les tables dans une seule
adult = pd.concat([adult1, adult2, adult34], ignore_index=True)

# dimension de la table adult
adult.shape

(194, 7)

### II.3. Nétoyage des données

Ici, on va appliquer des opérations de nétoyage. C'est à vous de déviner quelle opération a-t-on utilisé et pourqoi.


In [ ]:
# Afficher le nombre des valeurs nulles dans chaque colonne
adult.isnull().sum()

age                5
workclass         10
education          1
marital-status     4
sex                2
hours-per-week     2
class              0
dtype: int64

In [ ]:
adult.dropna(subset=["workclass", "education", "marital-status", "sex", "hours-per-week", "class"], inplace=True)
adult.isnull().sum()

age               3
workclass         0
education         0
marital-status    0
sex               0
hours-per-week    0
class             0
dtype: int64

**Analyse** 
- Quelle opération a-t-on appliqué ?
- Pourquoi ? (Quel est l'intérêt ?)

**Réponse**
- Suppression des lignes qui contiennent des valeurs nulles pour au moins une des colonnes: "workclass", "education", "marital-status", "sex", "hours-per-week", "class".
- Plusieurs models de machine learning ne fonctionnent pas avec des données manquantes. De plus, ça peut construire des modèles biaisés ce qui va conduire à des résultats faussés. L'analyse statistique peut manquer de précision à cause du manque de données (NaN).

In [ ]:
adult["age"] = pd.to_numeric(adult["age"])
adult["age"] = adult.groupby(["class", "education"])["age"].transform(lambda x: x.fillna(int(round(x.mean()))))
adult.isnull().sum()

age               0
workclass         0
education         0
marital-status    0
sex               0
hours-per-week    0
class             0
dtype: int64

**Analyse** 
- Quelle opération a-t-on appliqué ?
- Pourquoi ? (Quel est l'intérêt ?)

**Réponse**
- Remplacer les valeurs NaN de "age" pour chaque ligne avec la moyenne de l'age des individus ayant la même classe et education. (Constante selon la classe de l'individu)
- Comme cité précedemment, on traite les valeurs manquantes pour les mèmes raisons, cependant au lieu de les supprimer, on les remplace par la moyenne des ages des individus appartenant a une catégorie (classe et education) semblant avoir un age localement plus rapproché que la moyenne globale. Le remplaçement par une constante telque la moyenne n'impacte pas la variable et permet de ne pas refaire le calcul (ou dans notre cas recontacter l'individu pour demander son age) qui est une tâche fastidueuse, et ça permet de ne pas perdre les valeurs des autres attributs de cet individu parce qu'on ne le supprime pas à cause de la valeur manquante.

### II.4. Transformation des données

In [ ]:
adult["education"].head(6)

0    Bachelors
1    Bachelors
2      HS-grad
4    Bachelors
5      Masters
6          9th
Name: education, dtype: object

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
ord_enc = OrdinalEncoder()
# le résultat c'est un numpy.ndarray
education_enc = ord_enc.fit_transform(adult[["education"]])
education_enc[:6,]

array([[6.],
       [6.],
       [8.],
       [6.],
       [9.],
       [3.]])

**Analyse** 
- Quel est le type d'encodage utilisé ?
- A votre avis, dans quel cas peut-on utiliser ce type d'encodage ?

**Réponse**
- Encodage ordinal
- Quand les valeurs d'une variable qualitative peuvent être ordonnées (c'est une variable ordinale), on opte pour l'encodage ordinal pour garder la relation d'ordre et permettre au modèle de mieux l'apprendre. Pour ce cas, une relation d'ordre existe entre les niveau de scolarisation (licence, master, lycée..)

In [ ]:
adult["sex"].head(6)

0    M
1    M
2    M
4    F
5    F
6    F
Name: sex, dtype: object

In [ ]:
from sklearn.preprocessing import OneHotEncoder
onehot_enc = OneHotEncoder()
# le résultat c'est un numpy.ndarray
sex_enc = onehot_enc.fit_transform(adult[["sex"]])
sex_enc.toarray()[:6,]

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.]])

**Analyse** 
- Quel est le type d'encodage utilisé ?
- A votre avis, dans quel cas peut-on utiliser ce type d'encodage ?

**Réponse**
- Encodage OneHot
- Pour le cas d'une variable qualitative sans relation d'ordre, l'encodage Ordinal fausse les résultats du modèle en créant des inférences qui ne doivent pas exister à cause de la relation d'ordre qu'il impose. D'où la nécessité d'utiliser l'encodage OneHot qui n'impose rien aucune relation d'ordre sur les catégories.

In [ ]:
adult["hours-per-week"] = pd.to_numeric(adult["hours-per-week"])
adult["hours-per-week"].head(3)

0    40.0
1    13.0
2    40.0
Name: hours-per-week, dtype: float64

In [ ]:
from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler()
# le résultat c'est un numpy.ndarray
hours_per_week_prop = min_max_scaler.fit_transform(adult[["hours-per-week"]])
hours_per_week_prop[:3,]

array([[0.49367089],
       [0.15189873],
       [0.49367089]])

In [ ]:
# pour ajouter la nouvelle caractéristique au dataframe
adult["hours-per-week-prop"] = hours_per_week_prop
adult.head(3)

,age,workclass,education,marital-status,sex,hours-per-week,class,hours-per-week-prop
0,39.0,State-gov,Bachelors,single,M,40.0,N,0.493671
1,50.0,Self-emp-not-inc,Bachelors,married,M,13.0,N,0.151899
2,38.0,Private,HS-grad,divorced,M,40.0,N,0.493671


**Analyse** 
- Comment la normalisation MinMax est calculée ?
- Décrire les valeurs résultats (plage de valeurs, etc.) ?
- Est-ce que les valeurs du dataset de test sont garanties d'être dans la plage ?
- Si oui, expliquer pouruoi. Si non, comment garantir la plage des valeurs ?

**Réponse**
- Formule de MinMax: $minmax(X_j) = \frac{X_j - min(X_j)}{max(X_j) - min(X_j)}$ On soustrait le min et on divise par la difference entre le max et le min (la taille de l'intervalle des valeurs)
- Plage de valeurs: [0,1] de réels, avec 6 chiffres aprés la virgule (avec la representation de pandas). Cette méthode ne change pas la distribution des données dont l'effet des bruits n'est pas attenué et ils doivent être traités séparements.
- Non, elles ne sont pas garanties d'être dans la plage. Il se peut qu'une des valeurs de test soit supérieure au max ou inférieure au min.
- Pour garantir la plage des valeurs, il faut utiliser le max et min théoriques de la variable hours-per-week et non pas ceux du dataset de l'entraînement. Le min sera donc 0 hours-per-week, et le max sera 24*7 hours-per-week.

In [ ]:
adult["age"].head(3)

0    39.0
1    50.0
2    38.0
Name: age, dtype: float64

In [ ]:
from sklearn.preprocessing import StandardScaler

std_scaler = StandardScaler()
# le résultat c'est un numpy.ndarray
age_normal = std_scaler.fit_transform(adult[["age"]])
age_normal[:5,]

array([[ 0.111269  ],
       [ 0.99962632],
       [ 0.03050924],
       [-0.77708832],
       [-0.05025052]])

**Analyse** 
- Comment la normalisation standard est calculée ?
- Décrire les valeurs résultats (plage de valeurs, etc.) ?

**Réponse**
- La formule de normalisation Standard: $standard(X_j) = \frac{X_j - \mu(X_j)}{\sigma(X_j)}$ , On soustrait la moyenne et on divise par l'écart type.
- Les valeurs sont de type réel, 8 chiffres aprés la virgule (selon la représentation de pandas), sans bornes. Sachant que la variable centrée reduite suit une distribution normale N[0;1]: 68.2% des valeurs seront entre -1 et 1; 95.4% des valeurs seront entre -2 et 2; et 99.7% des valeurs seront entre -3 et 3...
De plus cette méthode reduit l'effet des bruits (on change la distribution de la variable)

In [ ]:
adult["age"].head(10)

0     39.0
1     50.0
2     38.0
4     28.0
5     37.0
6     49.0
7     52.0
8     31.0
9     42.0
10    43.0
Name: age, dtype: float64

In [ ]:
from sklearn.preprocessing import Binarizer

binarizer = Binarizer(threshold=40)
# le résultat c'est un numpy.ndarray
age_bin = binarizer.fit_transform(adult[["age"]])
age_bin[:10,]

array([[0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.]])

**Analyse** 
- Quelle est l'opération appliquée ici ?
- Quel est son rôle ?

**Réponse**
- l'Encodage Binary sur la colonne d'age, si c'est supérieur à 40 on met 1, et 0 sinon
- Cet encodage est utile quand les détails de la valeur d'une variable n'importent pas mais juste qu'elle soit en dessous ou en dessus d'un seuil donné. Elle est simple à implementer, et ne consomme pas beaucoup de mémoire, réduit considerablement la plage/cardinal de valeurs et réduit le nombre de dimensions par rapport au OneHot encoding mais on perd de l'information.